# Image Processing Pipeline — Interactive Examples

This notebook demonstrates how to use the **Image Processing Pipeline** with two modes:

| Mode | Description |
|------|-------------|
| **Full** | Load → FFC → Spatial Calibration → BEMD → Enhancement → Output |
| **PACE** | Load → BEMD → Enhancement → Output *(skip FFC & calibration)* |

All configuration parameters are editable in-place — just change the values in the cells below and re-run.

---

## Open in Colab

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Madeena-software/pace_implementation/blob/main/examples.ipynb)

## 1. Setup & Installation

In [ ]:
# If running on Google Colab, clone the repo first
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !git clone https://github.com/Madeena-software/pace_implementation.git
    os.chdir("pace_implementation")
    !pip install -q numpy opencv-python-headless cupy-cuda12x matplotlib
else:
    # Local — make sure we're in the project root
    os.chdir(os.path.dirname(os.path.abspath("__file__")))

print("Working directory:", os.getcwd())

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd())

from image_pipeline import ImageProcessingPipeline, PipelineConfig
from image_pipeline import (
    FlatFieldCorrection,
    SpatialCalibration,
    BEMD,
    HomomorphicFilter,
    NonlinearFilter,
    ImageEnhancer,
    ImageMetrics,
    ImageResizer,
)
import cv2
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from pathlib import Path

print("All imports OK ✓")

---
## 2. Configuration

Edit the values below to match your setup, then run the cell.

### 2a. Image Paths

In [ ]:
#@title Image Paths { run: "auto" }

# === Projection image (required for both modes) ===
proj_img_path = ""  #@param {type:"string"}

# === Gain / Dark / Calibration (only needed in Full mode) ===
gain_img_path = "datacitra/Gain/Bed/80_50_0,50.mdn"  #@param {type:"string"}
dark_img_path = "datacitra/Dark/Bed/dark.mdn"  #@param {type:"string"}
calibration_path = "datacitra/Kalibrasi/bed_44_35.npz"  #@param {type:"string"}

# === Output ===
output_dir = "output"  #@param {type:"string"}

print("Image paths configured ✓")

### 2b. BEMD Parameters

In [ ]:
#@title BEMD Parameters { run: "auto" }

bemd_max_iterations = 1  #@param {type:"integer"}
bemd_threshold = 1.0  #@param {type:"number"}
bemd_initial_window_size = 32  #@param {type:"integer"}
bemd_local_extrema_count = 10  #@param {type:"integer"}

print("BEMD parameters configured ✓")

### 2c. Enhancement Parameter Search Ranges

In [ ]:
#@title Enhancement Parameter Search Ranges { run: "auto" }

# Homomorphic filter
d0_values = [20, 30, 40]  # Cutoff frequencies
rh_values = [1.5, 2.0, 2.5]  # High-frequency gain
rl_values = [0.3, 0.5]  # Low-frequency gain

# Gamma correction
gamma_values = [0.8]

# CLAHE
clip_limit_values = [3.0]
tile_grid_size_values = [(8, 8)]

# Total combinations that will be evaluated
import itertools
n = len(list(itertools.product(d0_values, rh_values, rl_values, gamma_values, clip_limit_values, tile_grid_size_values)))
print(f"Enhancement parameters configured ✓  ({n} combinations)")

### 2d. Output & Performance Settings

In [ ]:
#@title Output & Performance { run: "auto" }

output_width = 4096  #@param {type:"integer"}
num_threads = 8  #@param {type:"integer"}
ffc_median_filter_size = 7  #@param {type:"integer"}

print("Output settings configured ✓")

### 2e. Build PipelineConfig object

This cell assembles all the settings above into a single `PipelineConfig`.

In [ ]:
config = PipelineConfig(
    proj_img_path=proj_img_path,
    gain_img_path=gain_img_path,
    dark_img_path=dark_img_path,
    calibration_path=calibration_path,
    output_dir=output_dir,
    ffc_median_filter_size=ffc_median_filter_size,
    bemd_max_iterations=bemd_max_iterations,
    bemd_threshold=bemd_threshold,
    bemd_initial_window_size=bemd_initial_window_size,
    bemd_local_extrema_count=bemd_local_extrema_count,
    d0_values=d0_values,
    rh_values=rh_values,
    rl_values=rl_values,
    gamma_values=gamma_values,
    clip_limit_values=clip_limit_values,
    tile_grid_size_values=tile_grid_size_values,
    output_width=output_width,
    num_threads=num_threads,
)

print("PipelineConfig ready ✓")
print(config)

---
## 3. Full Pipeline (FFC + Spatial Calibration + BEMD + Enhancement)

Requires: `proj_img_path`, `gain_img_path`, `dark_img_path`, `calibration_path`

In [ ]:
config.processing_mode = "full"
pipeline = ImageProcessingPipeline(config)
result_full = pipeline.process(show_plot=True)

print("\n" + "=" * 50)
print("FULL PIPELINE — RESULTS")
print("=" * 50)
print(f"Best parameters: {result_full.parameters}")
print(f"CII:         {result_full.cii:.4f}")
print(f"Entropy:     {result_full.entropy:.4f}")
print(f"EME:         {result_full.eme:.4f}")
print(f"Total Score: {result_full.total_score:.4f}")

---
## 4. PACE Mode (BEMD + Enhancement only — no FFC / calibration)

Requires only: `proj_img_path`

In [ ]:
config.processing_mode = "pace"
pipeline_pace = ImageProcessingPipeline(config)
result_pace = pipeline_pace.process(show_plot=True, mode="pace")

print("\n" + "=" * 50)
print("PACE PIPELINE — RESULTS")
print("=" * 50)
print(f"Best parameters: {result_pace.parameters}")
print(f"CII:         {result_pace.cii:.4f}")
print(f"Entropy:     {result_pace.entropy:.4f}")
print(f"EME:         {result_pace.eme:.4f}")
print(f"Total Score: {result_pace.total_score:.4f}")

---
## 5. Batch Processing — Full Pipeline

In [ ]:
#@title Batch settings (Full) { run: "auto" }

batch_input_dir = "datacitra/Thorax"  #@param {type:"string"}
batch_output_dir = "output"  #@param {type:"string"}

In [ ]:
config.processing_mode = "full"
pipeline_batch = ImageProcessingPipeline(config)

batch_results_full = []
for filename in os.listdir(batch_input_dir):
    if filename.lower().endswith((".tiff", ".tif", ".mdn")):
        print(f"\nProcessing: {filename}")
        p = os.path.join(batch_input_dir, filename)
        o = os.path.join(batch_output_dir, Path(filename).stem + "_processed.tiff")
        r = pipeline_batch.process(proj_path=p, output_path=o, show_plot=False)
        batch_results_full.append({"file": filename, "score": r.total_score, "params": r.parameters})

print("\n" + "=" * 50)
print("BATCH (FULL) — SUMMARY")
print("=" * 50)
for r in batch_results_full:
    print(f"{r['file']}: Score = {r['score']:.4f}")

---
## 6. Batch Processing — PACE Mode

In [ ]:
#@title Batch settings (PACE) { run: "auto" }

pace_batch_input_dir = "datacitra/Thorax"  #@param {type:"string"}
pace_batch_output_dir = "output"  #@param {type:"string"}

In [ ]:
config.processing_mode = "pace"
pipeline_pace_batch = ImageProcessingPipeline(config)

batch_results_pace = []
for filename in os.listdir(pace_batch_input_dir):
    if filename.lower().endswith((".tiff", ".tif", ".mdn")):
        print(f"\nProcessing (PACE): {filename}")
        p = os.path.join(pace_batch_input_dir, filename)
        o = os.path.join(pace_batch_output_dir, Path(filename).stem + "_pace_processed.tiff")
        r = pipeline_pace_batch.process(proj_path=p, output_path=o, show_plot=False, mode="pace")
        batch_results_pace.append({"file": filename, "score": r.total_score, "params": r.parameters})

print("\n" + "=" * 50)
print("BATCH (PACE) — SUMMARY")
print("=" * 50)
for r in batch_results_pace:
    print(f"{r['file']}: Score = {r['score']:.4f}")

---
## 7. Step-by-Step Processing (Full Pipeline)

Use this section when you need fine-grained control over each processing step.

In [ ]:
#@title Step-by-step: single-set filter params { run: "auto" }

step_proj_path = ""  #@param {type:"string"}
step_gain_path = "datacitra/Gain/Bed/80_50_0,50.mdn"  #@param {type:"string"}
step_dark_path = "datacitra/Dark/Bed/dark.mdn"  #@param {type:"string"}
step_calib_path = "datacitra/Kalibrasi/bed_44_35.npz"  #@param {type:"string"}

step_d0 = 30  #@param {type:"integer"}
step_rh = 2.0  #@param {type:"number"}
step_rl = 0.5  #@param {type:"number"}
step_gamma = 0.8  #@param {type:"number"}
step_clip_limit = 3.0  #@param {type:"number"}
step_output_width = 4096  #@param {type:"integer"}

In [ ]:
# Step 1: Load images
print("Step 1: Loading images...")
proj_img = cv2.imread(step_proj_path, -1)
gain_img = cv2.imread(step_gain_path, -1)
dark_img = cv2.imread(step_dark_path, -1)

# Step 2: Flat Field Correction
print("Step 2: Applying Flat Field Correction...")
ffc = FlatFieldCorrection(median_filter_size=ffc_median_filter_size)
ffc_image = ffc.apply(proj_img, gain_img, dark_img)

# Step 3: Spatial Calibration
print("Step 3: Applying Spatial Calibration...")
calibrator = SpatialCalibration(step_calib_path)
calibrated_image = calibrator.apply(ffc_image)

# Step 4: BEMD Decomposition
print("Step 4: BEMD Decomposition...")
bemd = BEMD(
    max_iterations=bemd_max_iterations,
    threshold=bemd_threshold,
    initial_window_size=bemd_initial_window_size,
    local_extrema_count=bemd_local_extrema_count,
)
bimfs = bemd.decompose(cp.asarray(calibrated_image))
energies = BEMD.calculate_energies(bimfs)

# Step 5: Homomorphic Filter
print("Step 5: Applying Homomorphic Filter...")
hf = HomomorphicFilter(d0=step_d0, rh=step_rh, rl=step_rl)
filtered_image = hf.apply(calibrated_image)

# Step 6: Nonlinear Filter (Denoise)
print("Step 6: Denoising...")
nlf = NonlinearFilter(r=1, beta=0.5)
denoised_image = nlf.denoise(bimfs, energies, filtered_image)

# Step 7: Gamma Correction
print("Step 7: Applying Gamma Correction...")
gamma_image = ImageEnhancer.gamma_correction(denoised_image, gamma=step_gamma)

# Step 8: CLAHE
print("Step 8: Applying CLAHE...")
clahe_image = ImageEnhancer.apply_clahe(gamma_image, clip_limit=step_clip_limit)

# Step 9: Metrics
print("Step 9: Calculating Metrics...")
mask = np.ones_like(calibrated_image)
cii = ImageMetrics.calculate_cii(clahe_image, calibrated_image, mask)
entropy = ImageMetrics.calculate_entropy(clahe_image)
eme = ImageMetrics.calculate_eme(clahe_image, 4, 4)

print(f"\nMetrics:")
print(f"  CII:     {cii:.4f}")
print(f"  Entropy: {entropy:.4f}")
print(f"  EME:     {eme:.4f}")

# Step 10: Resize & Save
print("Step 10: Resizing and saving...")
final_image = ImageResizer.resize(clahe_image, step_output_width).get()
os.makedirs("output", exist_ok=True)
cv2.imwrite("output/step_by_step.tiff", final_image.astype(np.uint16))

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(proj_img, cmap="gray"); axes[0].set_title("Original"); axes[0].axis("off")
axes[1].imshow(calibrated_image, cmap="gray"); axes[1].set_title("Calibrated"); axes[1].axis("off")
axes[2].imshow(final_image, cmap="gray"); axes[2].set_title("Processed"); axes[2].axis("off")
plt.tight_layout()
plt.show()
print("\nProcessing complete!")

---
## 8. Load Configuration from JSON

If you prefer managing settings via `config.json`, use this cell instead of sections 2a–2d.

In [ ]:
json_config_path = "config.json"  #@param {type:"string"}

config_from_json = PipelineConfig.from_json(json_config_path)
print("Loaded config from JSON ✓")
print(config_from_json)

In [ ]:
# --- Run full pipeline from JSON config ---
config_from_json.processing_mode = "full"  # change to "pace" for PACE mode

pipeline_json = ImageProcessingPipeline(config_from_json)
result_json = pipeline_json.process(show_plot=True)

print(f"\nBest parameters: {result_json.parameters}")
print(f"Total Score:     {result_json.total_score:.4f}")